In [ ]:
import pandas as pd
import requests
import json
from tqdm import tqdm
import time

from selenium import webdriver
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium_stealth import stealth
# import undetected_chromedriver as uc


chrome_options = Options()
# chrome_options.add_argument("--headless")  # Enable headless mode
chrome_options.add_argument("--no-sandbox")  # Required for some environments
chrome_options.add_argument("--disable-dev-shm-usage")  # Helps in Docker environments

from bs4 import BeautifulSoup

driver = webdriver.Chrome(options=chrome_options)
stealth(driver,
        # user_agent='your-custom-user-agent',  # Set custom user agent
        languages=["en-US", "en"],  # Set languages
        vendor="Mozilla",  # Set vendor
        platform="Win32",  # Set platform
        webgl_vendor="Mozilla",  # Set WebGL vendor
        renderer="Mozilla",  # Set WebGL renderer
        fix_hairline=True)  # Fix hairline issue in headless mode
driver.get('https://www.nseindia.com/market-data/52-week-low-equity-market')
cookies = driver.get_cookies()
time.sleep(2)
print(len(cookies))

In [75]:
# cookies = []
# while(len(cookies)<10):
#     driver.get('https://www.nseindia.com/market-data/52-week-low-equity-market')
#     time.sleep(2)
#     cookies = driver.get_cookies()
#     print(len(cookies))

In [76]:
cookie = ""
for item in driver.get_cookies():
    cookie += item['name'] + "=" + item['value'] + "; "

In [77]:
# specific_cookie = driver.get_cookie('_ga') 
# _ga = specific_cookie['value']

# specific_cookie = driver.get_cookie('nsit')
# nsit = specific_cookie['value']

# specific_cookie = driver.get_cookie('ak_bmsc') 
# ak_bmsc = specific_cookie['value']

# specific_cookie = driver.get_cookie('AKA_A2') 
# AKA_A2 = specific_cookie['value']

# specific_cookie = driver.get_cookie('nseappid') 
# nseappid = specific_cookie['value']

# specific_cookie = driver.get_cookie('bm_sz') 
# bm_sz = specific_cookie['value']

# specific_cookie = driver.get_cookie('RT') 
# RT = specific_cookie['value']


# specific_cookie = driver.get_cookie('_ga_87M7PJ3R97') 
# _ga_87M7PJ3R97 = specific_cookie['value']


# # specific_cookie = driver.get_cookie('_ga_WM2NSQKJEK') 
# # _ga_WM2NSQKJEK = specific_cookie['value']


# specific_cookie = driver.get_cookie('_abck') 
# _abck = specific_cookie['value']

# bm_sv =  driver.get_cookie('bm_sv')['value']

# cookie = f'defaultLang=en; _ga={_ga}; nsit={nsit}; AKA_A2=A; ak_bmsc={ak_bmsc}; nseQuoteSymbols=[{{"symbol":"TATASTEEL","identifier":null,"type":"equity"}}]; nseappid={nseappid}; bm_sz={bm_sz}; RT={RT}; _ga_87M7PJ3R97={_ga_87M7PJ3R97}; _ga_WM2NSQKJEK={_ga_WM2NSQKJEK}; _abck={_abck}; bm_sv={bm_sv}'

In [78]:

url = "https://www.nseindia.com/api/live-analysis-data-52weeklowstock"

payload = {}
headers = {
  'accept': '*/*',
  'accept-language': 'en-US,en;q=0.9',
  'cookie':cookie,
  'priority': 'u=1, i',
  'referer': 'https://www.nseindia.com/market-data/52-week-low-equity-market',
  'sec-fetch-dest': 'empty',
  'sec-fetch-mode': 'cors',
  'sec-fetch-site': 'same-origin',
  'user-agent': 'your-custom-user-agent'
}

response = requests.request("GET", url, headers=headers, data=payload)
data = json.loads(response.text)['data']
row = []
for indx in range(1,len(data)):
  symbol = data[indx]['symbol']
  price = data[indx]['ltp']
  name = data[indx]['comapnyName']
  row.append([symbol,price,name])

df = pd.DataFrame(row , columns=['symbol','price','name'])



In [79]:
symbol_to_name_dict = df.set_index('symbol')['name'].to_dict()

In [80]:


def get_reports(symbol , name):
  url = f"https://www.nseindia.com/api/corporates-financial-results?index=equities&symbol={symbol}&issuer={name.replace(' ','%20')}&period=Annual"

  payload = {}
  headers = {
    'accept': '*/*',
    'accept-language': 'en-GB,en-US;q=0.9,en;q=0.8',
    'cookie': cookie,
    'priority': 'u=1, i',
    'referer': 'https://www.nseindia.com/companies-listing/corporate-filings-financial-results?symbol=20MICRONS&tabIndex=equity',
    'sec-ch-ua': '"Chromium";v="128", "Not;A=Brand";v="24", "Opera";v="114"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"macOS"',
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'same-origin',
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/128.0.0.0 Safari/537.36 OPR/114.0.0.0'
  }

  response = requests.request("GET", url, headers=headers, data=payload)
  global report_df
  data = json.loads(response.text)
  # Create a new row to append
  new_row = pd.DataFrame([[symbol, name, data]], columns=['symbol','name','data'])

  # Concatenate the new row to the existing DataFrame
  report_df = pd.concat([report_df, new_row], ignore_index=True)

In [ ]:
report_df = pd.DataFrame()
# Loop with tqdm to show progress
for key, value in tqdm(symbol_to_name_dict.items(), desc="Processing symbols"):
    get_reports(key, value)

In [ ]:
report_df

In [83]:
report_df['found'] = ''
report_df['found'] = report_df.apply(lambda x: False if len(x['data'])==0 else True,axis=1)

In [ ]:
report_df['eps'] = ''
Non_Consolidated = pd.DataFrame()
for i in tqdm(range(report_df.shape[0])):
    if (report_df['found'][i]):
        temp = pd.DataFrame(report_df['data'][i])
        temp = temp[temp['consolidated']!='Non-Consolidated']
        urls = [item for item in temp['xbrl'].to_list() if item.rsplit('/',1)[-1]!='-']
        eps = []
        for link in urls:
            driver.get(link)
            
            # Parse the XML with BeautifulSoup
            soup = BeautifulSoup(driver.page_source, 'xml')

            # Define the namespace prefix
            namespace = {'in-bse-fin': 'http://example.com/ns'}

            # Find the tag with contextRef="FourD" by using the full qualified name and contextRef attribute
            tag = soup.find('in-bse-fin:BasicEarningsLossPerShareFromContinuingAndDiscontinuedOperations',
                            attrs={'contextRef': 'FourD'})


            # Extract the value of the tag if found
            value = tag.text.strip() if tag else None
            
            if value == None:
                tag = soup.find('in-bse-fin:DilutedEarningsPerShareAfterExtraordinaryItems',
                            attrs={'contextRef': 'FourD'})
                value = tag.text.strip() if tag else None
                

            eps.append(value)
        
        report_df.at[i,'eps'] = eps
        
        
        temp = pd.DataFrame(report_df['data'][i])
        temp = temp[temp['consolidated']=='Non-Consolidated']
        urls = [item for item in temp['xbrl'].to_list() if item.rsplit('/',1)[-1]!='-']
        eps = []
        for link in urls:
            driver.get(link)
            
            # Parse the XML with BeautifulSoup
            soup = BeautifulSoup(driver.page_source, 'xml')

            # Define the namespace prefix
            namespace = {'in-bse-fin': 'http://example.com/ns'}

            # Find the tag with contextRef="FourD" by using the full qualified name and contextRef attribute
            tag = soup.find('in-bse-fin:BasicEarningsLossPerShareFromContinuingAndDiscontinuedOperations',
                            attrs={'contextRef': 'FourD'})


            # Extract the value of the tag if found
            value = tag.text.strip() if tag else None
            
            if value == None:
                tag = soup.find('in-bse-fin:DilutedEarningsPerShareAfterExtraordinaryItems',
                            attrs={'contextRef': 'FourD'})
                value = tag.text.strip() if tag else None
                

            eps.append(value)
            
        copy_df = report_df.loc[i:i+1].copy(deep=True)
            
        Non_Consolidated = pd.concat([Non_Consolidated,copy_df],ignore_index=True)
        Non_Consolidated.at[Non_Consolidated.shape[0]-1,'eps'] = eps        
        
        
        
    

In [85]:
# import httpx

# url = "https://nsearchives.nseindia.com/corporate/xbrl/INDAS_118115_1365654_30012025020837.xml"

# headers = {
#     "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)",
#     "Referer": "https://www.nseindia.com/",
# }

# cookies = cookie

# with httpx.Client() as client:
#     response = client.get(url, headers=headers)

#     if response.status_code == 200:
#         with open("downloaded_file.xml", "wb") as file:
#             file.write(response.content)
#         print("File downloaded successfully!")
#     else:
#         print(f"Failed to download. Status Code: {response.status_code}")

In [86]:
report_df['consolidated'] = 'yes'
Non_Consolidated['consolidated'] = 'no'
report_df = pd.concat([report_df,Non_Consolidated],ignore_index=True)
report_df.sort_values(by='symbol',inplace=True)
report_df.reset_index(drop=True,inplace=True)


In [87]:
report_df.to_excel('LOW.xlsx',index=False)

In [88]:
driver.quit()

In [ ]:
report_df

In [90]:
report_df['avg_eps'] = report_df.apply(
    lambda x: sum(
        [float(item) for item in x['eps'] if item is not None and isinstance(item, (int, float,str))]
    ) / len(
        [float(item) for item in x['eps'] if item is not None and isinstance(item, (int, float ,str))]
    ) if len([float(item) for item in x['eps'] if item is not None and isinstance(item, (int, float ,str))]) > 0 else "",
    axis=1
)

In [91]:
report_df['total_eps'] = report_df.apply(
    lambda x: len([float(item) for item in x['eps'] if item is not None and isinstance(item, (int, float, str))]),
    axis=1
)

In [93]:
report_df['3Yr_avg_eps'] = report_df.apply(
    lambda x: sum(
        [float(item) for item in x['eps'] if item is not None and isinstance(item, (int, float,str))][:3]
    ) / 3 if len([float(item) for item in x['eps'] if item is not None and isinstance(item, (int, float,str))]) >= 3 else "", 
    axis=1
)

In [95]:
seven_and_above = report_df[report_df['total_eps']>=7].copy(deep=True)
three = report_df[(report_df['3Yr_avg_eps']!="")].copy(deep=True)
three = three[(three['3Yr_avg_eps']>0)].copy(deep=True)


In [96]:
three['fifftee_times']  = ""
three['fifftee_times'] = three.apply(lambda x: 15*x['3Yr_avg_eps'],axis=1)

In [97]:
def get_price(symbol):
    url = f"https://www.nseindia.com/api/quote-equity?symbol={symbol}"

    payload = {}
    headers = {
    'accept': '*/*',
    'accept-language': 'en-GB,en-US;q=0.9,en;q=0.8',
    'cookie': cookie,
    'priority': 'u=1, i',
    'referer': 'https://www.nseindia.com/get-quotes/equity?symbol=RELIANCE',
    'sec-ch-ua': '"Chromium";v="130", "Google Chrome";v="130", "Not?A_Brand";v="99"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"macOS"',
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'same-origin',
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/130.0.0.0 Safari/537.36'
    }

    response = requests.request("GET", url, headers=headers, data=payload)

    return json.loads(response.text)['priceInfo']['lastPrice']


In [98]:
driver.quit()

In [99]:
three.reset_index(drop=True,inplace=True)
three['cmp'] = ""

In [ ]:
for i in tqdm(range(three.shape[0])):
    three.loc[i,'cmp'] = get_price(three.loc[i,'symbol'])

In [101]:
three['cmp'] = three['cmp'].apply(lambda x: float(x))

In [ ]:
three[three['fifftee_times']>three['cmp']].sort_values(by=['cmp'],inplace=True)

In [ ]:
three[three['fifftee_times']>three['cmp']].sort_values(by=['cmp'])

In [104]:
# import requests
# from bs4 import BeautifulSoup

# links = []
# url = "https://www.moneycontrol.com/stocks/marketstats/highlows/index.php"

# payload = {}
# headers = {
#   'Cookie': 'PHPSESSID=s5ml7lgj149orlsq9pas210ac3'
# }

# response = requests.request("GET", url, headers=headers, data=payload)

# # Parse the HTML content
# soup = BeautifulSoup(response.text, 'html.parser')


# # Find the div with class "FL stat_lft columnst"
# target_div = soup.find("div", class_="FL stat_lft columnst")

# if target_div:
#     hrefs = [a['href'] for a in target_div.find_all('a', href=True)]
#     for link in hrefs:
#         links.append(link)
# else:
#     print("Target div not found.")
    
    
# stocks_name = []
# base_url = 'https://www.moneycontrol.com'
# for link in links:
#     link = base_url + link
#     link = link.replace('week-high','week-low')
#     response = requests.request("GET", link, headers=headers, data=payload)
#     soup = BeautifulSoup(response.text, 'html.parser')
#     target_div = soup.find_all("span", class_="gld13 disin")
    
#     for div in target_div:
#         text = div.get_text(strip=True)
#         a_tag = div.find("a")
#         href = a_tag['href'] if a_tag and 'href' in a_tag.attrs else 'N/A'
#         stocks_name.append(text)
    
